# Meeting Bot (Zoom) - Clean Colab Notebook

This notebook collects the required details first, then installs the repo, starts the server, and streams live server output. Use the stop cell to cleanly terminate the bot process group.

In [ ]:
# Edit the placeholders below, then run the rest of the notebook.
CONFIG = {
    "meetingId": "1234567890",
    "bearerToken": "YOUR_BEARER_TOKEN",
    "name": "Test Bot",
    "teamId": "TestTeam",
    "timezone": "UTC",
    "userId": "TestUser",
    "botId": "TEST_BOT_UUID",
    "webinarRegistration": {
        "firstName": "Raju",
        "lastName": "Rakesh",
        "email": "raju@myapp.co",
        "phone": "+91..."
    }
}

MEETING_URL = f"https://zoom.us/j/{CONFIG['meetingId']}"
API_ENDPOINT = "http://localhost:3000/zoom/join"

print("Config captured. Ready to setup and run.")

In [ ]:
%%bash
set -e

# Install system dependencies, including WebGL support and audio
apt-get update && \
  apt-get install -y \
  ffmpeg \
  libnss3 \
  libxss1 \
  libasound2 \
  pulseaudio \
  alsa-utils \
  libatk1.0-0 \
  libatk-bridge2.0-0 \
  libcups2 \
  libxkbcommon-x11-0 \
  libgbm-dev \
  libgl1-mesa-dri \
  libgl1-mesa-glx \
  mesa-utils \
  xvfb \
  wget \
  gnupg \
  xorg \
  xserver-xorg \
  libx11-dev \
  libxext-dev \
  dos2unix \
  imagemagick python3-pil \
  # ARM-specific packages (only for ARM64)
  $(if [ "$(dpkg --print-architecture)" = "arm64" ]; then echo "libgles2 libegl1 libwayland-egl1"; fi) \
  && rm -rf /var/lib/apt/lists/*

# Install Google Chrome (fallback to Chromium)
wget -q -O - https://dl.google.com/linux/linux_signing_key.pub | gpg --dearmor -o /usr/share/keyrings/google-linux-signing-key.gpg && \
  ARCH=$(dpkg --print-architecture) && \
  if [ "$ARCH" = "amd64" ]; then \
    echo "deb [arch=amd64 signed-by=/usr/share/keyrings/google-linux-signing-key.gpg] http://dl.google.com/linux/chrome/deb/ stable main" > /etc/apt/sources.list.d/google-chrome.list; \
  elif [ "$ARCH" = "arm64" ]; then \
    echo "deb [arch=arm64 signed-by=/usr/share/keyrings/google-linux-signing-key.gpg] http://dl.google.com/linux/chrome/deb/ stable main" > /etc/apt/sources.list.d/google-chrome.list; \
  else \
    echo "deb [signed-by=/usr/share/keyrings/google-linux-signing-key.gpg] http://dl.google.com/linux/chrome/deb/ stable main" > /etc/apt/sources.list.d/google-chrome.list; \
  fi && \
  apt-get update && apt-get install -y google-chrome-stable || apt-get install -y chromium && \
  rm -rf /var/lib/apt/lists/*

REPO_URL="https://github.com/smartm13/meeting-bot"
REPO_DIR="/content/meeting-bot"

if [ ! -d "$REPO_DIR" ]; then
  echo "Cloning repository..."
  git clone "$REPO_URL" "$REPO_DIR"
else
  echo "Repository already present."
fi

cd "$REPO_DIR"

npm install
npx playwright install --with-deps

dos2unix ./xvfb-run-wrapper
chmod +x ./xvfb-run-wrapper

npm run build

mkdir -p /tmp/.X11-unix
chmod 1777 /tmp/.X11-unix

mkdir -p "$REPO_DIR/dist/_tempvideo"
chmod -R 777 "$REPO_DIR/dist/_tempvideo"

mkdir -p ~/.config/pulse
chown -R "$(whoami)":"$(whoami)" ~/.config

npx playwright install

echo "Setup complete."

In [ ]:
import os
import queue
import signal
import subprocess
import threading
import time

BOT_PROCESS = None
BOT_LOG_QUEUE = queue.Queue()

def _log_reader(pipe, queue_obj):
    for line in iter(pipe.readline, ""):
        queue_obj.put(line)
    pipe.close()

def start_bot():
    global BOT_PROCESS
    if BOT_PROCESS and BOT_PROCESS.poll() is None:
        print("Bot already running.")
        return

    start_kwargs = {
        "cwd": REPO_DIR,
        "stdout": subprocess.PIPE,
        "stderr": subprocess.STDOUT,
        "text": True,
        "bufsize": 1
    }

    if os.name == "posix":
        start_kwargs["preexec_fn"] = os.setsid
    else:
        start_kwargs["creationflags"] = subprocess.CREATE_NEW_PROCESS_GROUP

    BOT_PROCESS = subprocess.Popen(["npm", "start"], **start_kwargs)
    threading.Thread(target=_log_reader, args=(BOT_PROCESS.stdout, BOT_LOG_QUEUE), daemon=True).start()
    print(f"Bot started. PID={BOT_PROCESS.pid}")

def stream_logs(poll_seconds=0.25, stop_line_prefix=None):
    print("Streaming bot output. Stop this cell to stop streaming.")
    while BOT_PROCESS and BOT_PROCESS.poll() is None:
        try:
            line = BOT_LOG_QUEUE.get(timeout=poll_seconds)
            print(line, end="")
            if stop_line_prefix and line.startswith(stop_line_prefix):
                print("\nStop line detected; ending log stream.")
                break
        except queue.Empty:
            pass
    while not BOT_LOG_QUEUE.empty():
        print(BOT_LOG_QUEUE.get(), end="")
    print("\nBot process ended.")

def stop_bot():
    global BOT_PROCESS
    if not BOT_PROCESS:
        print("Bot not running.")
        return
    if BOT_PROCESS.poll() is None:
        print("Stopping bot process group...")
        try:
            if os.name == "posix":
                os.killpg(os.getpgid(BOT_PROCESS.pid), signal.SIGTERM)
            else:
                BOT_PROCESS.terminate()
            BOT_PROCESS.wait(timeout=10)
        except Exception:
            if os.name == "posix":
                os.killpg(os.getpgid(BOT_PROCESS.pid), signal.SIGKILL)
            else:
                BOT_PROCESS.kill()
    BOT_PROCESS = None
    print("Bot stopped.")

start_bot()

In [ ]:
# Run this cell to continuously show live server output
stream_logs(stop_line_prefix="Server is running on")

In [ ]:
import requests

print("Waiting for server to initialize...")
time.sleep(15)

payload = {
    "bearerToken": CONFIG["bearerToken"],
    "url": MEETING_URL,
    "name": CONFIG["name"],
    "teamId": CONFIG["teamId"],
    "timezone": CONFIG["timezone"],
    "userId": CONFIG["userId"],
    "botId": CONFIG["botId"]
}

webinar_registration = CONFIG.get("webinarRegistration")
if webinar_registration:
    payload["webinarRegistration"] = webinar_registration

headers = {"Content-Type": "application/json"}
print(f"Sending POST request to {API_ENDPOINT} ...")
response = requests.post(API_ENDPOINT, json=payload, headers=headers)
print(f"Status: {response.status_code}")
try:
    print("Response JSON:", response.json())
except ValueError:
    print("Response text:", response.text)

print("Bot join request sent.")

In [ ]:
# Live Xvfb screen output (requires ImageMagick + Xvfb)
import time
from PIL import Image
from IPython.display import display, clear_output
import os
import subprocess

os.environ["DISPLAY"] = os.environ.get("DISPLAY", ":99")

while True:
    subprocess.run(["import", "-window", "root", "/tmp/xvfb.png"], check=True)
    clear_output(wait=True)
    display(Image.open("/tmp/xvfb.png"))
    time.sleep(2)


In [ ]:
# Stop the bot and clean up
stop_bot()